In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/FashioNet3/

/content/drive/MyDrive/FashioNet3


In [ ]:
!pip install albumentations

In [ ]:
!pip install ultralytics

In [ ]:
import os
import json
import torch
import cv2
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import albumentations as A
import random

In [ ]:
def apply_augmentations(image, keypoints):
    height, width = image.shape[:2]  # 이미지 높이, 너비 가져오기

    augmentations = A.Compose([
        A.HorizontalFlip(p=1.0),  # ✅ 항상 좌우 반전 적용
        A.RandomBrightnessContrast(p=0.5),
        A.GaussianBlur(p=0.3),
        A.Affine(scale=(0.8, 1.2), translate_percent=(-0.2, 0.2), rotate=(-45, 45), p=0.9, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 강한 변형 적용
        A.Perspective(scale=(0.1, 0.2), p=0.8, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 원근 변형 강하게 적용
        A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.7),  # ✅ 비선형 변형 적용
    ], keypoint_params=A.KeypointParams(format='xy', remove_invisible=True))

    augmented = augmentations(image=image, keypoints=keypoints)

    # ✅ keypoints 개수 검증 (손실 여부 확인)
    if len(augmented["keypoints"]) != len(keypoints):
        print(f"⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용")
        augmented["keypoints"] = keypoints  # 원본 keypoints 유지

    # ✅ YOLO 포맷으로 정규화 (0~1 범위)
    normalized_keypoints = [(x / width, y / height) for (x, y) in augmented["keypoints"]]
    normalized_keypoints = [(np.clip(x, 0, 1), np.clip(y, 0, 1)) for x, y in normalized_keypoints]

    print(f"🔍 원본 keypoints 개수: {len(keypoints)} → 증강 후 keypoints 개수: {len(augmented['keypoints'])} (변형됨)")
    return augmented["image"], normalized_keypoints

In [ ]:
class FashionDataset(Dataset):
    def __init__(self, image_files, json_files, json_folder, images_folder, img_size=640, split="train", augment=False):
        self.img_size = img_size
        self.json_folder = json_folder
        self.images_folder = images_folder
        self.split = split
        self.augment = augment  # ✅ 증강 여부 추가

        self.output_folder = os.path.join("fashion_dataset", "labels", self.split)
        os.makedirs(self.output_folder, exist_ok=True)

        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),
        ])

        self.image_files, self.json_files = self.match_files(image_files, json_files)
        self.classes = self.create_classes_and_polygons()
        self.class_to_index = {cls: idx for idx, cls in enumerate(self.classes)}

    # .jpg, .json 매칭
    def match_files(self,image_files, json_files):
      img_dict = {os.path.splitext(os.path.basename(f))[0].lower(): f for f in image_files}
      json_dict = {os.path.splitext(os.path.basename(f))[0].lower(): f for f in json_files}

      matched_images = []
      matched_jsons = []
      unmatched_images = []
      unmatched_jsons = []

      for key in img_dict.keys():
        if key in json_dict:
            matched_images.append(img_dict[key])
            matched_jsons.append(json_dict[key])
        else:
            unmatched_images.append(img_dict[key])

      print(f"✅ 매칭된 이미지 개수: {len(matched_images)}, 매칭된 JSON 개수: {len(matched_jsons)}")

      return matched_images, matched_jsons

    def create_classes_and_polygons(self):
        """
        YOLOv8 형식의 세그멘테이션 라벨(.txt) 파일 생성.
        모든 좌표는 0~1로 정규화됨.
        """
        unique_classes = set()
        valid_images = set()

        # ✅ 클래스 수집
        for json_file in self.json_files:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)

            try:
                labeling = data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"]
            except KeyError:
                continue

            for category in ["아우터", "하의", "원피스", "상의"]:
                if category in labeling and labeling[category]:
                    for item in labeling[category]:
                        if "카테고리" in item and item["카테고리"]:
                            class_name = f"{category}-{item['카테고리']}"
                            unique_classes.add(class_name)

        # ✅ 클래스 정렬 및 매핑 생성
        sorted_classes = sorted(unique_classes)  # 클래스 이름을 정렬
        class_mapping = {class_name: class_id for class_id, class_name in enumerate(sorted_classes)}

        print(f"📌 {self.split} 데이터의 고유 클래스 목록: {sorted_classes}")

        # ✅ YOLO 형식의 라벨 생성
        for json_file in self.json_files:
            with open(json_file, "r", encoding="utf-8") as f:
                data = json.load(f)

            try:
                labeling = data["데이터셋 정보"]["데이터셋 상세설명"]["라벨링"]
                polygon_coords = data["데이터셋 정보"]["데이터셋 상세설명"]["폴리곤좌표"]
                image_info = data["이미지 정보"]
                img_w = image_info["이미지 너비"]
                img_h = image_info["이미지 높이"]
            except KeyError:
                continue

            image_name = os.path.basename(json_file).replace(".json", ".jpg")
            txt_file_path = os.path.join(self.output_folder, image_name.replace(".jpg", ".txt"))

            polygon_found = False  # 폴리곤 좌표가 있는지 체크

            with open(txt_file_path, "w", encoding="utf-8") as txt_file:
                for category in ["아우터", "하의", "원피스", "상의"]:
                    if category in labeling and labeling[category]:
                        for item in labeling[category]:
                            if "카테고리" in item and item["카테고리"]:
                                class_name = f"{category}-{item['카테고리']}"
                                class_id = class_mapping[class_name]

                                if category in polygon_coords and polygon_coords[category]:
                                    for polygon in polygon_coords[category]:
                                        polygon_found = True

                                        # ✅ 폴리곤 좌표 정렬 및 정규화
                                        x_keys = sorted([k for k in polygon.keys() if k.startswith("X좌표")], key=lambda x: int(x.replace("X좌표", "")))
                                        y_keys = sorted([k for k in polygon.keys() if k.startswith("Y좌표")], key=lambda y: int(y.replace("Y좌표", "")))

                                        coords_list = [(polygon[x_keys[i]] / img_w, polygon[y_keys[i]] / img_h) for i in range(len(x_keys))]

                                        # ✅ YOLO 형식으로 저장
                                        line_parts = [str(class_id)]
                                        for (xn, yn) in coords_list:
                                            line_parts.append(f"{xn:.6f}")
                                            line_parts.append(f"{yn:.6f}")

                                        txt_file.write(" ".join(line_parts) + "\n")

            if not polygon_found:
                print(f"⚠️ 폴리곤이 없는 경우 .txt 파일 유지됨 (빈 파일로 생성됨): {txt_file_path}")
                open(txt_file_path, 'w').close()
            else:
                print(f"✅ {self.split} 데이터에서 .txt 파일 생성됨: {txt_file_path}")
                valid_images.add(image_name)

        print(f"🎯 {self.split} 데이터의 YOLOv8 세그멘테이션 라벨이 .txt 파일로 생성 완료!")

        return sorted_classes  # ✅ 반환값 추가

    def __getitem__(self, index):
        image_path = self.image_files[index]
        image_name = os.path.basename(image_path)

        txt_file_path = os.path.join(self.output_folder, image_name.replace(".jpg", ".txt"))
        augmented_image_path = image_path
        augmented_label_path = txt_file_path

        labels = []
        keypoints = []

        # ✅ YOLO 라벨 로드
        if os.path.exists(txt_file_path):
            with open(txt_file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
                class_ids = [int(line.strip().split()[0]) for line in lines]

            should_augment = any(cid in [6, 7, 9, 11, 12, 15] for cid in class_ids) and 14 not in class_ids
            if should_augment:
                for line in lines:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    polygon = [float(p) for p in parts[1:]]

                    if len(polygon) % 2 == 0:  # ✅ (x, y) 좌표 쌍이어야 함
                        labels.append(torch.tensor([class_id] + polygon))
                        keypoints.append([(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)])

        # ✅ 데이터 증강 수행
        if should_augment and keypoints:
            print(f"✅ 즉시 증강 시작: {image_path}")
            image_rgb = Image.open(image_path).convert("RGB")
            image_np = np.array(image_rgb)

            keypoints_flat = [kp for sublist in keypoints for kp in sublist]  # 폴리곤 좌표 flatten

            for i in range(20):
              augmented_image, augmented_keypoints = apply_augmentations(image_np, keypoints_flat)

              augmented_image_path = os.path.join("/content/drive/MyDrive/FashioNet3/fashion_dataset/images/train", image_name.replace(".jpg", f"_aug_{i}.jpg"))
              augmented_label_path = os.path.join("/content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train", image_name.replace(".jpg", f"_aug_{i}.txt"))

              os.makedirs(os.path.dirname(augmented_image_path), exist_ok=True)
              os.makedirs(os.path.dirname(augmented_label_path), exist_ok=True)

    # ✅ 증강된 이미지 저장
              cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    # ✅ YOLO 형식으로 좌표 변환 및 저장
              if augmented_keypoints:
                polygon_str = ' '.join([f"{coord:.6f}" for point in augmented_keypoints for coord in point])
                with open(augmented_label_path, "w", encoding="utf-8") as f:
                  f.write(f"{class_id} {polygon_str}\n")
                print(f"✅ 증강된 라벨 저장 완료: {augmented_label_path}")
              else:
                print(f"⚠️ Warning: 증강 후 keypoints가 비어 있어 라벨 저장하지 않음 → {augmented_label_path}")

        return image_path, txt_file_path, augmented_image_path, augmented_label_path

    def __len__(self):
        return len(self.image_files)

In [ ]:
import glob

def load_dataset():
    image_files = glob.glob("/content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/*.jpg")
    label_files = glob.glob("/content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/*.txt")
    json_files = glob.glob("/content/drive/MyDrive/FashioNet3/labels/train/*.json")

    # ✅ 원본 및 증강된 데이터 포함
    augmented_images = glob.glob("/content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/*_aug_*.jpg")
    augmented_labels = glob.glob("/content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/*_aug_*.txt")

    image_files.extend(augmented_images)
    label_files.extend(augmented_labels)

    print(f"📌 총 이미지 개수 (증강 포함): {len(image_files)}")
    print(f"📌 총 라벨 개수 (증강 포함): {len(label_files)}")

    return image_files, label_files, json_files

In [ ]:
from torch.utils.data import DataLoader, WeightedRandomSampler

image_files, label_files, json_files = load_dataset()

train_dataset = FashionDataset(
    image_files=image_files,
    json_files = json_files,
    json_folder="/content/drive/MyDrive/FashioNet3/labels/train",
    images_folder="/content/drive/MyDrive/FashioNet3/fashion_dataset/images/train",
    img_size=640,
    split="train",
    augment=True
)

class_counts = [741, 21, 1397, 1199, 336, 1607, 131, 304, 173, 899,
                729, 78, 510, 276, 2267, 131, 36, 1254, 98, 1190, 1914]
total_samples = sum(class_counts)

weights = [total_samples / count for count in class_counts]
sample_labels = [train_dataset.class_to_index[label] for label in train_dataset.classes]
sample_weights = [weights[label] for label in sample_labels]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

# dataset 로드 성공 여부 체크
data_iter = iter(train_dataloader)
try:
    image_path, txt_file_path, augmented_image_path, augmented_label_path = next(data_iter)
    print(f"첫 번째 배치 로드 성공!")
    print(f"원본 이미지 경로: {image_path}")
    print(f"원본 라벨 경로: {txt_file_path}")
    print(f"증강 이미지 경로: {augmented_image_path}")
    print(f"증강 라벨 경로: {augmented_label_path}")
except StopIteration:
    print("❌ DataLoader에서 데이터 로드 실패! 데이터셋이 비어 있음.")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36508.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36484.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36520.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36519.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36515.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36487.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36496.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36486.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36497.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36495.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36498.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36488.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36482.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_dataset/labels/train/36505.txt
✅ train 데이터에서 .txt 파일 생성됨: fashion_datas

<ipython-input-38-66f48d05c468>:8: UserWarning: Argument(s) 'cval, mode' are not valid for transform Affine
  A.Affine(scale=(0.8, 1.2), translate_percent=(-0.2, 0.2), rotate=(-45, 45), p=0.9, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 강한 변형 적용
<ipython-input-38-66f48d05c468>:9: UserWarning: Argument(s) 'cval, mode' are not valid for transform Perspective
  A.Perspective(scale=(0.1, 0.2), p=0.8, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 원근 변형 강하게 적용
<ipython-input-38-66f48d05c468>:8: UserWarning: Argument(s) 'cval, mode' are not valid for transform Affine
  A.Affine(scale=(0.8, 1.2), translate_percent=(-0.2, 0.2), rotate=(-45, 45), p=0.9, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 강한 변형 적용
<ipython-input-38-66f48d05c468>:10: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.7),  # ✅ 비선형 변형 적용
<ipython-input-38-66f48d05c468>:9: UserWarning: Argument(s) 'cval, mode' are not valid for

✅ 즉시 증강 시작: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/163908.jpg
⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용


<ipython-input-38-66f48d05c468>:8: UserWarning: Argument(s) 'cval, mode' are not valid for transform Affine
  A.Affine(scale=(0.8, 1.2), translate_percent=(-0.2, 0.2), rotate=(-45, 45), p=0.9, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 강한 변형 적용



🔍 원본 keypoints 개수: 48 → 증강 후 keypoints 개수: 48 (변형됨)

<ipython-input-38-66f48d05c468>:9: UserWarning: Argument(s) 'cval, mode' are not valid for transform Perspective
  A.Perspective(scale=(0.1, 0.2), p=0.8, cval=(0,0,0), mode=cv2.BORDER_CONSTANT),  # ✅ 원근 변형 강하게 적용


🔍 원본 keypoints 개수: 66 → 증강 후 keypoints 개수: 66 (변형됨)



<ipython-input-38-66f48d05c468>:10: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=1, sigma=50, alpha_affine=50, p=0.7),  # ✅ 비선형 변형 적용


✅ 증강된 라벨 저장 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/163954_aug_0.txt
✅ 증강된 라벨 저장 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/17828_aug_0.txt
⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용
🔍 원본 keypoints 개수: 51 → 증강 후 keypoints 개수: 51 (변형됨)
⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용
🔍 원본 keypoints 개수: 48 → 증강 후 keypoints 개수: 48 (변형됨)
⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용
🔍 원본 keypoints 개수: 66 → 증강 후 keypoints 개수: 66 (변형됨)
✅ 증강된 라벨 저장 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/163908_aug_0.txt
✅ 증강된 라벨 저장 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/163954_aug_1.txt
✅ 증강된 라벨 저장 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/17828_aug_1.txt
⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용
🔍 원본 keypoints 개수: 48 → 증강 후 keypoints 개수: 48 (변형됨)
⚠️ Warning: keypoints 유실됨! 원본 keypoints 사용
🔍 원본 keypoints 개수: 51 → 증강 후 keypoints 개수: 51 (변형됨)
✅ 증강된 라벨 저장 완료: /content/drive/MyDrive/FashioNe

In [ ]:
import os

# train 폴더에 저장된 일부 라벨 파일 확인
label_folder = "fashion_dataset/labels/train"
label_files = sorted([f for f in os.listdir(label_folder) if f.endswith(".txt")])

print(f"라벨 파일 개수: {len(label_files)}")

if len(label_files) > 0:
    sample_file = os.path.join(label_folder, label_files[0])
    with open(sample_file, "r", encoding="utf-8") as f:
        lines = f.readlines()

    print(f"{sample_file} 내용:")
    print("\n".join(lines))

✅ 라벨 파일 개수: 11311
✅ fashion_dataset/labels/train/1.txt 내용:
10 0.396250 0.005020 0.296250 0.082520 0.222500 0.308770 0.278750 0.411270 0.293750 0.701270 0.356250 0.722520 0.402500 0.483770 0.376250 0.252520 0.408750 0.158770 0.390000 0.117520 0.403750 0.032519 0.456250 0.041270 0.462500 0.101270 0.440000 0.152520 0.497500 0.320020 0.488750 0.716270 0.545000 0.657520 0.563750 0.395020 0.618750 0.290020 0.591250 0.182520 0.562500 0.092520 0.532500 0.055019 0.475000 0.030019 0.460000 0.020019 0.401250 0.002520

19 0.382500 0.310020 0.368750 0.723770 0.341250 0.728770 0.352500 0.795020 0.396250 0.803770 0.420000 0.788770 0.415000 0.821270 0.477500 0.817520 0.502500 0.697520 0.502500 0.388770 0.497500 0.307520 0.391250 0.307520

5



In [ ]:
import os
import random
import glob

# 데이터 경로 설정
image_folder = "/content/drive/MyDrive/FashioNet3/fashion_dataset/images/train"
label_folder = "/content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train"

# 클래스 14가 포함된 라벨 파일 찾기
class_14_labels = []
for label_file in glob.glob(os.path.join(label_folder, "*.txt")):
    with open(label_file, "r", encoding="utf-8") as f:
        lines = f.readlines()
        class_ids = [int(line.strip().split()[0]) for line in lines]
        if len(set(class_ids)) == 1 and 14 in class_ids:
            class_14_labels.append(label_file)

# 1,000개만 랜덤 선택하여 삭제
num_to_delete = min(1000, len(class_14_labels))
delete_labels = random.sample(class_14_labels, num_to_delete)

# 이미지와 라벨 삭제
deleted_count = 0
for label_file in delete_labels:
    image_file = os.path.join(image_folder, os.path.basename(label_file).replace(".txt", ".jpg"))

    try:
        if os.path.exists(label_file):
            os.remove(label_file)  # 라벨 삭제
        if os.path.exists(image_file):
            os.remove(image_file)  # 이미지 삭제
        deleted_count += 1
        print(f"삭제 완료: {image_file}, {label_file}")
    except Exception as e:
        print(f"삭제 실패: {image_file}, {label_file} → {e}")

print(f"클래스 14의 데이터 {deleted_count}개 삭제 완료!")

🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/15291.jpg, /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/15291.txt
🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/27952.jpg, /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/27952.txt
🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/14280.jpg, /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/14280.txt
🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/450998.jpg, /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/450998.txt
🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/44864.jpg, /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/44864.txt
🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/11516.jpg, /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train/11516.txt
🗑️ 삭제 완료: /content/drive/MyDrive/FashioNet3/fashion_data

In [ ]:
# 클래스별 데이터 개수 구하기
import os
from collections import defaultdict

# labels 폴더 경로 설정
label_folder = "fashion_dataset/labels/train"

# 클래스별 파일 개수를 저장할 딕셔너리 (0~20까지 초기화)
class_counts = {i: 0 for i in range(21)}

# .txt 파일 탐색
label_files = [f for f in os.listdir(label_folder) if f.endswith(".txt")]

# 각 파일에서 class_id 개수 확인
for label_file in label_files:
    file_path = os.path.join(label_folder, label_file)

    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # 해당 파일에서 등장하는 클래스 ID를 저장할 집합 (중복 방지)
    found_classes = set()

    for line in lines:
        parts = line.strip().split()
        if parts:  # 빈 줄 방지
            class_id = int(parts[0])  # 첫 번째 값이 class_id

            if 0 <= class_id <= 20:  # 클래스 ID 범위 확인
                found_classes.add(class_id)

    # 해당 파일에 등장한 class_id별로 개수 증가
    for class_id in found_classes:
        class_counts[class_id] += 1

# 결과 출력
for class_id, count in class_counts.items():
    print(f"클래스 {class_id}: {count}개 파일")

클래스 0: 741개 파일
클래스 1: 21개 파일
클래스 2: 1397개 파일
클래스 3: 1199개 파일
클래스 4: 336개 파일
클래스 5: 1607개 파일
클래스 6: 131개 파일
클래스 7: 304개 파일
클래스 8: 173개 파일
클래스 9: 899개 파일
클래스 10: 729개 파일
클래스 11: 78개 파일
클래스 12: 510개 파일
클래스 13: 276개 파일
클래스 14: 2267개 파일
클래스 15: 131개 파일
클래스 16: 36개 파일
클래스 17: 1254개 파일
클래스 18: 98개 파일
클래스 19: 1190개 파일
클래스 20: 1914개 파일


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

# Train

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from ultralytics import YOLO
import os
import json
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

model = YOLO("yolov8n-seg.pt")

model.train(
    data="/content/drive/MyDrive/FashioNet3/dataset.yaml",
    epochs = 30,
    imgsz = 640,
    batch = 16,
    name = "fashio_seg")

Ultralytics 8.3.78 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/drive/MyDrive/FashioNet3/dataset.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=fashio_seg3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

train: Scanning /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train... 11311 images, 86 backgrounds, 1339 corrupt: 100%|██████████| 11331/11331 [02:54<00:00, 65.04it/s] 

train: WARNING ⚠️ /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/1.jpg: ignoring corrupt image/label: zero-size array to reduction operation minimum which has no identity
train: WARNING ⚠️ /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/100308.jpg: ignoring corrupt image/label: zero-size array to reduction operation minimum which has no identity
train: WARNING ⚠️ /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/102.jpg: ignoring corrupt image/label: zero-size array to reduction operation minimum which has no identity
train: WARNING ⚠️ /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/103.jpg: ignoring corrupt image/label: zero-size array to reduction operation minimum which has no identity
train: WARNING ⚠️ /content/drive/MyDrive/FashioNet3/fashion_dataset/images/train/104871.jpg: ignoring corrupt image/label: zero-size array to reduction operation minimum which has no identity
train: WARNING ⚠️ /content/drive/MyDrive/FashioNet3

train: New cache created: /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/drive/MyDrive/FashioNet3/fashion_dataset/labels/val... 999 images, 4 backgrounds, 53 corrupt: 100%|██████████| 1000/1000 [02:28<00:00,  6.74it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:09<00:00,  3.28it/s]
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:474: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:474: UserWarning: Glyph 51032 (\N{HANGUL SYLLABLE YI}) missing from font(s) DejaVu Sans.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:474: UserWarning: Glyph 45768 (\N{HANGUL SYLLABLE NI}) missing from font(s) DejaVu Sans.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:474: UserWarning: Glyph 53944 (\N{HANGUL SYLLABLE TEU}) missing from font(s) DejaVu Sans.
  fig.savefig(save_dir, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics

                   all        947       1214     0.0887      0.753       0.13      0.107     0.0893      0.726       0.13     0.0967
               상의-니트웨어         39         39     0.0356      0.949       0.33      0.236     0.0346      0.923      0.328      0.258
                상의-브라탑        247        247      0.589      0.632       0.56      0.468      0.596       0.64      0.567       0.43
               상의-블라우스         14         14    0.00908      0.929     0.0323     0.0282    0.00908      0.929     0.0323     0.0235
                 상의-셔츠         30         30     0.0225      0.967     0.0526     0.0416     0.0209        0.9     0.0479     0.0355
                  상의-탑         43         43     0.0434      0.977        0.1     0.0837     0.0424      0.953     0.0998     0.0785
                상의-티셔츠         10         10    0.00548          1     0.0183     0.0157    0.00439        0.8      0.015     0.0119
                상의-후드티          7          7     0.0164          1   

/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:436: UserWarning: Glyph 49345 (\N{HANGUL SYLLABLE SANG}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:436: UserWarning: Glyph 51032 (\N{HANGUL SYLLABLE YI}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:436: UserWarning: Glyph 45768 (\N{HANGUL SYLLABLE NI}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:436: UserWarning: Glyph 53944 (\N{HANGUL SYLLABLE TEU}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-packages/ultralytics/utils/metrics.py:436: UserWarning: Glyph 50920 (\N{HANGUL SYLLABLE WE}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
/usr/local/lib/python3.11/dist-p

Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/segment/fashio_seg3


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x788be4344850>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037

In [ ]:
metrics = model.val(data="/content/drive/MyDrive/FashioNet2/dataset.yaml")

# Test 및 Cropping

In [ ]:
# 폴리곤 좌표대로 cropping
# 매개 변수 : image_path : 원본 이미지 경로 // polygon_points : 폴리곤 좌표 (test 결과 넣으면 될 듯)
def crop_by_polygon_mask(image_path, polygon_points):
  image = cv2.imread(image_path)
  if image is None:
    print(f"이미지 로드 실패: {image_path}")
    return None
  h, w = image.shape[:2]

  # BGR -> BGRA (알파 채널 추가)
  image_rgba = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

  # 마스 생성
  mask = np.zeros((h,w), dtype=np.uint8)

  # 폴리곤 좌표를 numpy 배열로 변환
  pts = np.array([polygon_points], dtype=np.int32)
  cv2.fillPoly(mask, pts, 255)

  # 마스크 적용
  image_rgba[:,:,3] = mask

  return image_rgba

In [ ]:
# test 및 image cropping
import cv2
import numpy as np
import torch
from torch.utils.data import DataLoader
from ultralytics import YOLO
import os
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image

model = YOLO("/content/drive/MyDrive/FashioNet3/runs/segment/fashio_seg3/weights/best.pt")  # 학습 완료 가중치
results = model.predict(source="/content/drive/MyDrive/FashioNet3/test_images",
                        save=False, show=False, conf=0.25, imgsz=640)

# 결과 저장
output_folder = "/content/drive/MyDrive/FashioNet3/output_mask"
cropped_folder = "/content/drive/MyDrive/FashioNet3/output_mask/cropped_images"
os.makedirs(output_folder, exist_ok=True)
os.makedirs(cropped_folder, exist_ok=True)

# 마스크가 그려진 이미지 저장 + 폴리곤 좌표 추출 (for cropping)
for result in results:
  annotated_img = result.plot()

  image_path = result.path
  filename = os.path.basename(image_path)

  save_path = os.path.join(output_folder, filename)
  cv2.imwrite(save_path, annotated_img)
  print(f"저장 완료:{save_path}")

  # 폴리곤 좌표 추출 및 image cropping
  polygon_points = []
  if result.masks is not None and len(result.masks.xy) > 0:
    polygon_points = [(int(x), int(y)) for (x,y) in result.masks.xy[0]]

  if len(polygon_points) > 2:
    rgb_image = crop_by_polygon_mask(image_path, polygon_points)

    png_filename = filename.replace(".jpg", "_mask.png")
    save_path_rgba = os.path.join(cropped_folder, png_filename)
    cv2.imwrite(save_path_rgba, rgb_image)
    print(f"저장 완료(RGBA mask) : {save_path_rgba}")

print("테스트 및 결과 이미지 저장 완료!")


image 1/21 /content/drive/MyDrive/FashioNet3/test_images/test01.jpg: 640x480 1 하의-스커트, 47.3ms
image 2/21 /content/drive/MyDrive/FashioNet3/test_images/test02.jpg: 640x480 1 원피스-드레스, 8.2ms
image 3/21 /content/drive/MyDrive/FashioNet3/test_images/test03.jpg: 640x480 1 상의-셔츠, 8.5ms
image 4/21 /content/drive/MyDrive/FashioNet3/test_images/test04.jpg: 640x480 (no detections), 8.3ms
image 5/21 /content/drive/MyDrive/FashioNet3/test_images/test05.jpg: 640x512 1 상의-셔츠, 1 하의-청바지, 46.8ms
image 6/21 /content/drive/MyDrive/FashioNet3/test_images/test06.jpg: 640x384 2 하의-팬츠s, 47.3ms
image 7/21 /content/drive/MyDrive/FashioNet3/test_images/test07.jpg: 640x480 1 원피스-드레스, 1 하의-스커트, 9.1ms
image 8/21 /content/drive/MyDrive/FashioNet3/test_images/test08.jpg: 640x544 2 상의-니트웨어s, 1 하의-청바지, 46.6ms
image 9/21 /content/drive/MyDrive/FashioNet3/test_images/test09.jpg: 640x480 1 상의-셔츠, 9.3ms
image 10/21 /content/drive/MyDrive/FashioNet3/test_images/test10.jpg: 640x480 (no detections), 8.3ms
image 11/21 /conten